## Convert old IMAS at NERSC to new IMAS 
June 21, 2024 

Venkitesh Ayyar

In [1]:
import numpy as np
import datetime
import os

import pymongo
import pprint
from bson.objectid import ObjectId

import yaml
import json
import xmltodict

In [2]:
# ## Convert json to yaml for easy viewing
# with open('gyrokinetics_new.json', 'r') as j:
#     GK_dict = json.loads(j.read())
    
# with open('gyrokinetics_new.yaml','w') as f:
#     yaml.dump(GK_dict,f,default_flow_style=False)

## Load both yaml files

In [3]:
fname='gyro_imas_old.yaml'
with open(fname,'r') as f:
    gk_dict = yaml.safe_load(f)
    
fname='gyro_imas_new.yaml'
with open(fname,'r') as f:
    GK_dict = yaml.safe_load(f)

## Print dictionary hierarchy

In [4]:
def f_print_dict_struct(dict1,tab_count=0):
    '''
    Module to print dictionary hierarchy
    '''
    
    if isinstance(dict1,dict):
        for i,j in dict1.items():
            if not isinstance(j,dict):
                print('\t'*tab_count, i,'\t',type(j))
                if (isinstance(j,list) and len(j)>0):
                    if isinstance(j[0],dict):
                        print('\t\t'*tab_count,i,'[0]:')                    
                        f_print_dict_struct(j[0],tab_count+2)
            else: 
                # print('\t'*tab_count,"------------")
                print('\t'*tab_count,i,':')
                f_print_dict_struct(j, tab_count+1)

                
# f_print_dict_struct(gk_dict,0)

# f_print_dict_struct(GK_dict,0)
    

In [5]:
def f_get_dict_all_keys(dict1):
    '''
    Module to get lowest keys in dict hierarchy
    '''
    keys_lst =[] 
    
    if isinstance(dict1,dict):
        for i,j in dict1.items():
            if not isinstance(j,dict):
                keys_lst.append(i)
                if (isinstance(j,list) and len(j)>0):
                    if isinstance(j[0],dict):
                        keys_lst.extend( f_get_dict_all_keys(j[0]) )
            else: 
                keys_lst.extend( f_get_dict_all_keys(j) ) 

    return keys_lst

# keys_par2 = f_get_dict_all_keys(par2)

In [6]:
gk_dict.keys(),GK_dict.keys()

(dict_keys(['code', 'collisions', 'flux_surface', 'fluxes_integrated_norm', 'ids_properties', 'model', 'species', 'species_all', 'wavevector']),
 dict_keys(['code', 'collisions', 'flux_surface', 'ids_properties', 'linear', 'max_repr_length', 'model', 'non_linear', 'normalizing_quantities', 'species', 'species_all', 'time', 'version']))

In [7]:
for key2 in GK_dict['model'].keys():
    if key2.startswith('include'):
        print(key2,GK_dict['model'][key2])

include_a_field_parallel 1
include_b_field_parallel 0
include_centrifugal_effects None
include_coriolis_drift None
include_full_curvature_drift None


In [8]:
for key2 in gk_dict['model'].keys():
    if key2.startswith('include'):
        print(key2,gk_dict['model'][key2])


include_a_field_parallel True
include_b_field_parallel False
include_centrifugal_effects False
include_full_curvature_drift False


In [9]:
collision_suffix = ['energy_conservation', 'finite_larmor_radius', 'momentum_conservation', 'pitch_only']
for suffix in collision_suffix:
    print(suffix,gk_dict['model']['collision_'+suffix])
    print(suffix,GK_dict['model']['collisions_'+suffix])


energy_conservation True
energy_conservation None
finite_larmor_radius False
finite_larmor_radius None
momentum_conservation True
momentum_conservation None
pitch_only False
pitch_only None


In [10]:
# # ## modification for species 
# key1='species'

# common_keys   = [i for i in gk_dict[key1][0].keys() if i in GK_dict[key1][0].keys()]
# old_only_keys = [i for i in gk_dict[key1][0].keys() if i not in GK_dict[key1][0].keys()]
# new_only_keys = [i for i in GK_dict[key1][0].keys() if i not in gk_dict[key1][0].keys()]

# print(common_keys,'\n\n',old_only_keys,'\n\n',new_only_keys)


In [11]:
for key2 in ['include_a_field_parallel', 'include_b_field_parallel', 'include_centrifugal_effects', 'include_full_curvature_drift'] :
    print(key2, gk_dict['model'][key2],GK_dict['model'][key2])

include_a_field_parallel True 1
include_b_field_parallel False 0
include_centrifugal_effects False None
include_full_curvature_drift False None


In [12]:
key1='model'

common_keys   = [i for i in gk_dict[key1].keys() if i in GK_dict[key1].keys()]
old_only_keys = [i for i in gk_dict[key1].keys() if i not in GK_dict[key1].keys()]
new_only_keys = [i for i in GK_dict[key1].keys() if i not in gk_dict[key1].keys()]

print(common_keys,'\n\n',old_only_keys,'\n\n',new_only_keys)

# for i in common_keys: 
#     print(i,'\t',gk_dict[key1][i],'\t',GK_dict[key1][i],'\n')

# for i in old_only_keys:
#     print(i, gk_dict[key1][i])
    
# for i in new_only_keys:
#     print(i, GK_dict[key1][i])

['include_a_field_parallel', 'include_b_field_parallel', 'include_centrifugal_effects', 'include_full_curvature_drift'] 

 ['collision_ei_only', 'collision_energy_conservation', 'collision_finite_larmor_radius', 'collision_momentum_conservation', 'collision_pitch_only', 'initial_value_run', 'non_linear_run', 'time_interval_norm'] 

 ['adiabatic_electrons', 'collisions_energy_conservation', 'collisions_finite_larmor_radius', 'collisions_momentum_conservation', 'collisions_pitch_only', 'include_coriolis_drift', 'max_repr_length', 'version']


In [13]:
# for key2 in  ['adiabatic_electrons', 'include_coriolis_drift', 'max_repr_length', 'version']:
#     print(key2, GK_dict['model'][key2])

for key2 in  ['collision_ei_only','initial_value_run', 'non_linear_run', 'time_interval_norm']: 
    print(key2, gk_dict['model'][key2])

collision_ei_only False
initial_value_run True
non_linear_run False
time_interval_norm []


### Wavevector

In [15]:
gk_dict['wavevector'][0].keys(), GK_dict['linear']['wavevector'][0].keys()

(dict_keys(['binormal_component_norm', 'eigenmode', 'poloidal_turns', 'radial_component_norm']),
 dict_keys(['binormal_wavevector_norm', 'eigenmode', 'max_repr_length', 'radial_wavevector_norm', 'version']))

In [16]:
key1='a_field_parallel_perturbed_norm_imaginary'
len(gk_dict['wavevector'][0]['eigenmode'][0][key1])

992

In [17]:
key1='a_field_parallel_perturbed_norm'
GK_dict['linear']['wavevector'][0]['eigenmode'][0]['fields'][key1].keys()
len(GK_dict['linear']['wavevector'][0]['eigenmode'][0]['fields'][key1]['__ndarray_tolist_real__'])
len(GK_dict['linear']['wavevector'][0]['eigenmode'][0]['fields'][key1]['__ndarray_tolist_real__'][0])
GK_dict['linear']['wavevector'][0]['eigenmode'][0]['fields'][key1]['dtype']
GK_dict['linear']['wavevector'][0]['eigenmode'][0]['fields'][key1]['shape']
# list of list : (384, 550)

[384, 550]

In [18]:
keys = ['frequency_norm','growth_rate_norm','growth_rate_tolerance']
for key in keys: 
    print(gk_dict['wavevector'][0]['eigenmode'][0][key],GK_dict['linear']['wavevector'][0]['eigenmode'][0][key])

0.24583409045936327 -1.6596400401476432
0.0474108603028772 -0.28141296468495464
0.001755957788995452 1.44941776692768


In [19]:
len(GK_dict['linear']['wavevector'][0]['eigenmode'][0]['angle_pol'])
len(gk_dict['wavevector'][0]['eigenmode'][0]['poloidal_angle'])



992

In [20]:
gk_dict['wavevector'][0]['eigenmode'][0]['moments_norm_rotating_frame'][0].keys()

dict_keys(['density_imaginary', 'density_real', 'temperature_parallel_imaginary', 'temperature_parallel_real', 'temperature_perpendicular_imaginary', 'temperature_perpendicular_real', 'velocity_parallel_imaginary', 'velocity_parallel_real'])

In [43]:
for i in GK_dict['linear']['wavevector'][0]['eigenmode'][0].keys():
    print(i)

angle_pol
code
fields
frequency_norm
growth_rate_norm
growth_rate_tolerance
initial_value_run
linear_weights
linear_weights_rotating_frame
max_repr_length
moments_norm_gyrocenter
moments_norm_gyrocenter_bessel_0
moments_norm_gyrocenter_bessel_1
moments_norm_particle
poloidal_turns
time_norm
version


In [32]:
# f_print_dict_struct(GK_dict['linear']['wavevector'][0]['eigenmode'][0],0)

In [79]:
GK_dict['linear']['wavevector'][0]['eigenmode'][0].keys()

dict_keys(['angle_pol', 'code', 'fields', 'frequency_norm', 'growth_rate_norm', 'growth_rate_tolerance', 'initial_value_run', 'linear_weights', 'linear_weights_rotating_frame', 'max_repr_length', 'moments_norm_gyrocenter', 'moments_norm_gyrocenter_bessel_0', 'moments_norm_gyrocenter_bessel_1', 'moments_norm_particle', 'poloidal_turns', 'time_norm', 'version'])

In [78]:
GK_dict['linear']['version']

{'idspy_internal_version': '2.0',
 'idspy_version': '034000.2.0',
 'imas_dd_git_commit': '845f1b30816f86a3cd4d53714dc56cdd307fdca1',
 'imas_dd_version': '03.40.00'}

In [80]:
GK_dict['linear']['wavevector'][0]['eigenmode'][0][']

dict_keys(['angle_pol', 'code', 'fields', 'frequency_norm', 'growth_rate_norm', 'growth_rate_tolerance', 'initial_value_run', 'linear_weights', 'linear_weights_rotating_frame', 'max_repr_length', 'moments_norm_gyrocenter', 'moments_norm_gyrocenter_bessel_0', 'moments_norm_gyrocenter_bessel_1', 'moments_norm_particle', 'poloidal_turns', 'time_norm', 'version'])

In [64]:
key='linear_weights'
key='fields'
GK_dict['linear']['wavevector'][0]['eigenmode'][0][key].keys()

dict_keys(['a_field_parallel_perturbed_norm', 'a_field_parallel_perturbed_parity', 'a_field_parallel_perturbed_weight', 'b_field_parallel_perturbed_norm', 'b_field_parallel_perturbed_parity', 'b_field_parallel_perturbed_weight', 'max_repr_length', 'phi_potential_perturbed_norm', 'phi_potential_perturbed_parity', 'phi_potential_perturbed_weight', 'version'])

In [31]:

def f_wavevector(old_gk_dict,new_gk_dict):
    ''' conversion for key: wavevector '''
    
    main_key = 'wavevector'
    old_gk,new_gk = old_gk_dict[main_key][0], new_gk_dict['linear'][main_key][0]
    
    dict1={}
    ## Old keys not present 
    # poloidal_turns is inside eigenmodes for new 
    
    ## New keys not present in old 
    keys = ['max_repr_length', 'version']
    
    for key in keys:
        dict1[key] = new_gk[key]
    
    ## Common keys 
    key_map = {'binormal_component_norm':'binormal_wavevector_norm', 
               'radial_component_norm':'radial_wavevector_norm' }
    for key,val in key_map.keys():
        dict1[val] = old_gk[key]

    ## Map subdict eigenmode 
    def f_get_eigenmode_dict(old_gk,new_gk):

        dict2={}

        ## Common keys
        keys = ['frequency_norm','growth_rate_norm','growth_rate_tolerance']
        for key in keys: 
            dict2[key]= old_gk['eigenmode'][0][key]

        ## Poloidal angle has different key name
        dict2['angle_pol'] = old_gk['eigenmode'][0]['poloidal_angle']

        ## poloidal_turns is outside eigenmodes in old 
        dict2['poloidal_turns'] = old_gk['poloidal_turns']
        
        ##### fluxes_norm
        dict2['linear_weights']={}

        ## Common keys 
        keys = ['energy_a_field_parallel', 'energy_b_field_parallel', 'energy_phi_potential', 'momentum_tor_parallel_a_field_parallel', 'momentum_tor_parallel_b_field_parallel', 
                'momentum_tor_parallel_phi_potential', 'momentum_tor_perpendicular_a_field_parallel', 'momentum_tor_perpendicular_b_field_parallel', 'momentum_tor_perpendicular_phi_potential', 
                'particles_a_field_parallel', 'particles_b_field_parallel', 'particles_phi_potential']

        num_list = 3 ## size of list : corresponds to 3 types of particles
        for key in keys: 
            dict2['linear_weights'][key]= [old_gk['fluxes_norm'][i][key] for i in range(num_list)]
        
        ## Keys not in old 
        keys = ['max_repr_length', 'version']
        for key in keys:
            dict2[key] = new_gk[key]

        ##### moments_norm_rotating_frame
        ## Not present in new IMAS 
        keys = ['density_imaginary', 'density_real', 'temperature_parallel_imaginary', \
                'temperature_parallel_real', 'temperature_perpendicular_imaginary', \
                'temperature_perpendicular_real', 'velocity_parallel_imaginary', 'velocity_parallel_real']

        ##### phi_potential_perturbed_norm 

        ##### a_field_parallel_perturbed_norm


        ### fields
        keys = ['a_field_parallel_perturbed_norm', 'a_field_parallel_perturbed_parity', 'a_field_parallel_perturbed_weight', 'b_field_parallel_perturbed_norm', 'b_field_parallel_perturbed_parity', 'b_field_parallel_perturbed_weight', 'max_repr_length', 'phi_potential_perturbed_norm', 'phi_potential_perturbed_parity', 'phi_potential_perturbed_weight', 'version']
        fields=dict.fromkeys(keys,None)
        dict2['fields']=fields
        
        ### moments_norm_particle 
        keys = ['density', 'heat_flux_parallel', 'j_parallel', 'max_repr_length', 'pressure_parallel', 'pressure_perpendicular', 'v_parallel_energy_perpendicular', 'v_perpendicular_square_energy', 'version']
        mom_norm = dict.fromkeys(keys,None)
        dict2['moments_norm_particle']=mom_norm
        
        ### Not present in old 
        keys = ['initial_value_run','linear_weights_rotating_frame','moments_norm_gyrocenter','moments_norm_gyrocenter_bessel_0','moments_norm_gyrocenter_bessel_1',]
        for key in keys: 
            dict2[key]= None
        
        for key in ['version','max_repr_length']
            dict2[key]= None
        
        ## time_norm present in model ??
        
        return [dict2] 

    ## eigenmode:
    dict1['eigenmode']= f_get_eigenmode_dict(old_gk,new_gk)

    dict1['eigenmode'][0]['time_norm'] = old_gk_dict['model']['time_interval_norm'] 

    ## Also need to add     keys = ['max_repr_length', 'version'] to main dict, outside of wavevector
        
    return dict1

### Code

In [22]:
gk_dict['code'].keys(), GK_dict['code'].keys()

(dict_keys(['name', 'parameters', 'version']),
 dict_keys(['commit', 'description', 'library', 'max_repr_length', 'name', 'output_flag', 'parameters', 'repository', 'version']))

In [23]:
gk_dict['code']['name']

'GENE'

####  Parameters 

In [24]:
GK_dict['code']['parameters']

'<?xml version="1.0" encoding="utf-8"?>\n<root><parallelization><n_procs_s>1</n_procs_s><n_procs_v>2</n_procs_v><n_procs_w>4</n_procs_w><n_procs_x>1</n_procs_x><n_procs_y>1</n_procs_y><n_procs_z>32</n_procs_z><n_procs_sim>256</n_procs_sim><n_parallel_sims>3</n_parallel_sims></parallelization><box><n_spec>3</n_spec><nx0>7</nx0><nky0>1</nky0><nz0>64</nz0><nv0>36</nv0><nw0>8</nw0><kymin>1.5</kymin><lv>3.0</lv><lw>9.0</lw><adapt_lx>true</adapt_lx><x0>0.825</x0><n0_global>32</n0_global><ky0_ind>1</ky0_ind></box><in_out><diagdir>/pscratch/sd/j/joeschm/NSXTU_discharges/132588/r_0.825_q5_MTM_mode/convergence_check/nz0_edgeopt_hypz_scans_V3/nz0-64_edge_opt-12_hyp_z--8/scanfiles0000/</diagdir><read_checkpoint>false</read_checkpoint><write_checkpoint>false</write_checkpoint><istep_field>100</istep_field><istep_mom>600</istep_mom><istep_nrg>50</istep_nrg><istep_omega>20</istep_omega><istep_vsp>5000</istep_vsp><istep_schpt>5000</istep_schpt><istep_energy>100</istep_energy><write_std>true</write_std

In [25]:
par1 = gk_dict['code']['parameters']
par1.keys()

dict_keys(['Bref', 'ENDIANNESS', 'ExBrate', 'GIT_BRANCH', 'GIT_MASTER', 'Lref', 'OMP_NUM_THREADS', 'Omega0_tor', 'PRECISION', 'RELEASE', 'Tref', 'adapt_lx', 'beta', 'calc_dt', 'charge1', 'charge2', 'charge3', 'coll', 'coll_cons_model', 'coll_order', 'collision_op', 'comp_type', 'debye2', 'dens1', 'dens2', 'dens3', 'diag_Blev(0)', 'diag_trap_levels', 'diagdir', 'dpdx_pm', 'dpdx_term', 'dt_max', 'dt_vlasov', 'ev_coll', 'ev_coll_est', 'geomfile', 'hyp_z', 'hypz_compensation', 'init_cond', 'init_time', 'istep_energy', 'istep_field', 'istep_mom', 'istep_nrg', 'istep_omega', 'istep_schpt', 'istep_vsp', 'kx_center', 'ky0_ind', 'kymin', 'lv', 'lw', 'lx', 'ly', 'magn_geometry', 'major_R', 'mass1', 'mass2', 'mass3', 'minor_r', 'mref', 'n0_global', 'n_fields', 'n_moms', 'n_parallel_sims', 'n_pol', 'n_procs_s', 'n_procs_sim', 'n_procs_v', 'n_procs_w', 'n_procs_x', 'n_procs_y', 'n_procs_z', 'n_spec', 'name1', 'name2', 'name3', 'nblocks', 'nky0', 'nltdt_off', 'nonlinear', 'norm_flux_projection', 'nr

In [26]:
# GK_dict['ids_properties']
par2 = xmltodict.parse(GK_dict['code']['parameters'])
par2['root'].keys()

dict_keys(['parallelization', 'box', 'in_out', 'general', 'geometry', 'species', 'info', 'units', 'bsgrid'])

In [27]:
[ky2 for ky1 in par2['root'].keys() for ky2 in par2['root'][ky1]]

['n_procs_s',
 'n_procs_v',
 'n_procs_w',
 'n_procs_x',
 'n_procs_y',
 'n_procs_z',
 'n_procs_sim',
 'n_parallel_sims',
 'n_spec',
 'nx0',
 'nky0',
 'nz0',
 'nv0',
 'nw0',
 'kymin',
 'lv',
 'lw',
 'adapt_lx',
 'x0',
 'n0_global',
 'ky0_ind',
 'diagdir',
 'read_checkpoint',
 'write_checkpoint',
 'istep_field',
 'istep_mom',
 'istep_nrg',
 'istep_omega',
 'istep_vsp',
 'istep_schpt',
 'istep_energy',
 'write_std',
 'iterdb_file',
 'nonlinear',
 'comp_type',
 'perf_vec',
 'nblocks',
 'arakawa_zv',
 'timescheme',
 'dt_max',
 'dt_vlasov',
 'ev_coll',
 'timelim',
 'ntimesteps',
 'beta',
 'debye2',
 'collision_op',
 'coll',
 'coll_cons_model',
 'init_cond',
 'hyp_z_with_dz_prefactor',
 'hyp_z',
 'hyp_v_with_dv_prefactor',
 'hyp_v',
 'perf_tsteps',
 'magn_geometry',
 'q0',
 'shat',
 'geomdir',
 'geomfile',
 'edge_opt',
 'minor_r',
 'major_r',
 'rhostar',
 'dpdx_term',
 'dpdx_pm',
 'norm_flux_projection',
 {'name': 'i',
  'omn': '5.9963121',
  'omt': '2.8994479',
  'mass': '1.0',
  'temp': '1.4

In [68]:
# len(par1.keys()),len(keys_par2)
# par2

In [29]:
common = [i for i in par1.keys() if i in keys_par2]
not_in_new = [i for i in par1.keys() if i not in keys_par2]
not_in_old = [i for i in keys_par2 if i not in par1.keys()]

NameError: name 'keys_par2' is not defined

In [ ]:
# len(common),len(not_in_new),len(not_in_old)

In [ ]:
# f_print_dict_struct(par2,0)

### Modules

In [ ]:
def f_collisions(old_gk_dict,new_gk_dict):
    ''' conversion for key: collisions '''

    main_key='collisions'
    old_gk,new_gk=old_gk_dict[main_key],new_gk_dict[main_key]
    
    dict1={}
    for key in ['max_repr_length','version']: 
        dict1[key] = new_gk[key]
    
    dict1['collisionality_norm']=old_gk['collisionality_norm']
    
    return dict1

def f_flux_surface(old_gk_dict,new_gk_dict):
    ''' conversion for key: flux_surface '''

    main_key='flux_surface'
    old_gk,new_gk=old_gk_dict[main_key],new_gk_dict[main_key]
    
    dict1={}
    ## Old keys not present 
    keys = ['triangularity_lower', 'triangularity_upper']
    
    ## New keys not present in old 
    keys = ['delongation_dr_minor_norm', 'dgeometric_axis_r_dr_minor', 'dgeometric_axis_z_dr_minor', 'max_repr_length', 'version']
    for key in keys:
        dict1[key] = new_gk[key]
    
    ## Common keys 
    common_keys = ['b_field_tor_sign', 'dc_dr_minor_norm', 'ds_dr_minor_norm', 'elongation', 'ip_sign', 'magnetic_shear_r_minor', 'pressure_gradient_norm', 'q', 'r_minor_norm', 'shape_coefficients_c', 'shape_coefficients_s']
    
    for key in common_keys:
        dict1[key] = old_gk[key]
        
    return dict1

def f_species_all(old_gk_dict,new_gk_dict):
    ''' conversion for key: species_all '''
    
    main_key='species_all'
    old_gk,new_gk=old_gk_dict[main_key],new_gk_dict[main_key]
    
    dict1={}
    ## Old keys not present 
    keys =  ['debye_length_reference', 'zeff'] 
    
    ## New keys not present in old 
    keys = ['angle_pol', 'debye_length_norm', 'max_repr_length', 'version']
    for key in keys:
        dict1[key] = new_gk[key]
    
    ## Common keys 
    common_keys = ['beta_reference', 'shearing_rate_norm', 'velocity_tor_norm'] 

    for key in common_keys:
        dict1[key] = old_gk[key]
        
    return dict1

def f_species(old_gk_dict, new_gk_dict):
    ''' conversion for key: species '''

    main_key='species'
    old_gk,new_gk=old_gk_dict[main_key][0],new_gk_dict[main_key][0]

    dict1={}
    ## Old keys not present 
    keys =  ['name'] 

    ## New keys not present in old 
    keys = ['max_repr_length', 'potential_energy_gradient_norm', 'potential_energy_norm', 'version']
    for key in keys:
        dict1[key] = new_gk[key]

    ## Common keys 
    common_keys = ['charge_norm', 'density_log_gradient_norm', 'density_norm', 'mass_norm', 'temperature_log_gradient_norm', 'temperature_norm', 'velocity_tor_gradient_norm'] 

    for key in common_keys:
        dict1[key] = old_gk[key]
    
    return [dict1]

def f_ids_properties(old_gk_dict,new_gk_dict):
    ''' conversion for key: ids_properties '''

    main_key='ids_properties'
    old_gk,new_gk=old_gk_dict[main_key],new_gk_dict[main_key]
    
    dict1={}
    ## Old keys not present 
    keys = ['creator', 'date'] 
    
    ## New keys not present in old 
    keys = ['creation_date', 'homogeneous_time', 'max_repr_length', 'name', 'occurrence_type', 'provenance', 'provider', 'version']
    for key in keys:
        dict1[key] = new_gk[key]
    
    ## Common keys 
    common_keys = ['comment'] 

    for key in common_keys[:]:
        dict1[key] = old_gk[key]
        
    ## Special fixes
    dict1['creation_data']=old_gk['date']
    dict1['provider'] = 'manual'
    
    ## Note: 'creator' not used. Should it be stored in 'name' ? 
    # dict1['name'] = old_gk['creator']

    return dict1

def f_model(old_gk_dict,new_gk_dict):
    ''' conversion for key: model '''

    ## To Do : Confirm what NoneType means for the True case in the new schema
    
    main_key='model'
    old_gk,new_gk=old_gk_dict[main_key],new_gk_dict[main_key]
    
    dict1={}
    ## Old keys not present 
    keys =  ['collision_ei_only','initial_value_run', 'non_linear_run', 'time_interval_norm']
    
    ## New keys not present in old 
    keys = ['adiabatic_electrons', 'include_coriolis_drift']
    for key in keys: 
        dict1[key] = None
    
    keys = ['max_repr_length', 'version']
    for key in keys:
        dict1[key] = new_gk[key]
    
    ## Common keys 
    common_keys = ['include_a_field_parallel', 'include_b_field_parallel']
    for key in common_keys: # Integer dtype
        dict1[key] = 1 if old_gk[key] else 0

    common_keys = ['include_centrifugal_effects', 'include_full_curvature_drift'] 
    for key in common_keys: ## None type
        dict1[key] = 1 if old_gk[key] else None
        
    ## common keys with slight modification in key name
    collision_suffix = ['energy_conservation', 'finite_larmor_radius', 'momentum_conservation', 'pitch_only']
    for suffix in collision_suffix: # old : collision_ (Bool) , new: collisions_ (None type)
        dict1['collisions_'+suffix] = 1 if gk_dict[main_key]['collision_'+suffix] else None 
        
    return dict1

# f_collisions(gk_dict,GK_dict)
# f_flux_surface(gk_dict,GK_dict)
# f_species_all(gk_dict,GK_dict)
# f_species(gk_dict,GK_dict)
# f_ids_properties(gk_dict,GK_dict)
f_model(gk_dict,GK_dict)

